In [1]:
import os
import time
import pathlib

import pandas as pd

import openai
os.environ['OPENAI_API_KEY'] = "sk-48characterstofakeanopenaikey48charactersopenai0"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "sk-48characterstofakeanopenaikey48charactersopenai0"
openai.api_base = "http://10.0.0.222:30307/v1"

# model used to retrieve answers
model = "Writer/camel-5b-hf"

# model used to generate the index
index_model = "Writer/camel-5b-hf"
persist_path = f"storage/{index_model.replace('/', '-')}-default-no-coref"


In [2]:
import logging
import sys

#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama-index/')

logging.basicConfig(stream=sys.stdout, level=logging.CRITICAL)

# Load KG from storage


In [3]:

from llama_index import StorageContext
from llama_index import ServiceContext
from llama_index import load_index_from_storage 
from llama_index.langchain_helpers.text_splitter  import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser

import tiktoken

#extensions to llama_index to support openai compatible endpoints, e.g. llama-api
from kron.llm_predictor.KronOpenAILLM import KronOpenAI
from kron.llm_predictor.KronLLMPredictor import KronLLMPredictor

from pyvis.network import Network

/home/arylwen/anaconda3/envs/ray39/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [4]:
#writer/camel uses endoftext 
from llama_index.utils import globals_helper
enc = tiktoken.get_encoding("gpt2")
tokenizer = lambda text: enc.encode(text, allowed_special={"<|endoftext|>"})
globals_helper._tokenizer = tokenizer

In [5]:
# define LLM - handles response format and <|endoftext|>
llm=KronOpenAI(temperature=0.01, model=model)
#for query we need more max tokens to carry the triples
llm.max_tokens = 1024 
# adds camel and mpt to the list of approved llms
llm_predictor = KronLLMPredictor(llm)
print(llm_predictor.metadata)

context_window=2048 num_output=1024 is_chat_model=False is_function_calling_model=False model_name='unknown'


In [6]:
# define TextSplitter
#192-48
text_splitter = SentenceSplitter(chunk_size=192, chunk_overlap=48, paragraph_separator='\n')
#define NodeParser
node_parser = SimpleNodeParser(text_splitter=text_splitter)
#define ServiceContext
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, node_parser=node_parser)

In [7]:
print(f'Loading index from {persist_path}')
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_path)
# load index
index = load_index_from_storage(storage_context=storage_context, 
                                    service_context=service_context, 
                                    max_triplets_per_chunk=2,
                                    show_progress = True)

Loading index from storage/Writer-camel-5b-hf-default-no-coref


In [8]:
ref_doc_info = index.ref_doc_info

In [9]:
ref_doc_info.keys()

dict_keys(['2205.10271v1.Compression_ensembles_quantify_aesthetic_complexity_and_the_evolution_of___visual_art', '2305.12027v1.Polar_Ducks_and_Where_to_Find_Them__Enhancing_Entity_Linking_with_Duck___Typing_and_Polar_Box_Embeddings', '2304.01852v3.Summary_of_ChatGPT_GPT_4_Research_and_Perspective_Towards_the_Future_of___Large_Language_Models', '2305.17346v1.Input_Aware_Dynamic_Timestep_Spiking_Neural_Networks_for_Efficient___In_Memory_Computing', '2304.03098v1.Static_Fuzzy_Bag_of_Words__a_lightweight_sentence_embedding_algorithm', '2303.01410v1.NLP_Workbench__Efficient_and_Extensible_Integration_of_State_of_the_art___Text_Mining_Tools', '2302.02023v1.TextShield__Beyond_Successfully_Detecting_Adversarial_Sentences_in_Text___Classification', '2306.00176v1.Automated_Annotation_with_Generative_AI_Requires_Validation', '2204.10817v3.Reward_Reports_for_Reinforcement_Learning', '2211.16412v1.Procedural_Image_Programs_for_Representation_Learning', '2005.14165v4.Language_Models_are_Few_Shot_Lea

In [10]:
ref_doc_info['2211.08732v2.Lesion_Guided_Explainable_Few_Weak_shot_Medical_Report_Generation']

RefDocInfo(node_ids=['2c2346d9-af03-4e52-a8c5-991adef40c59', '05dda7ee-5522-4052-a237-0807cae3bfd8', 'af77a838-04ec-4c0f-865b-985b981cee54', '9df3ddfe-783a-427a-927b-e0f31a7985fa', '70611b33-c930-45e9-bd88-2efa860204ad', '46136d3c-ce50-4b7f-9b1b-4503c7fda41f', '4522828d-ca39-4ddf-9311-35536cdb1386', '85b78aeb-2ac4-4ca4-8345-02a4285301d1', '36a0d4d2-5f59-4bb4-bc47-daa7246382ef', 'b4bb0c74-5a86-4d34-8c50-1f458b6672d3', 'eb47dd1f-3b9e-4940-958d-a9ea7ee9b707', '1722909b-f7f2-4193-8657-3f7ea739fa8b', '9ce70380-5968-4f6f-a355-c759732ae1e5', '5c15941f-4760-454a-afb3-f6e05f65e027', 'b8940fb2-f64e-49cc-8937-18698cc14e7e', '24b99598-ee4c-4c6b-aa50-ae1bdfb29ba6', '83e47648-6819-49b2-ac62-982bff704845', 'b9d29448-485d-4f89-b50c-2202ecc5ea11', 'e8de2aa8-0c54-4bb6-9f19-0164f8f881c3', '1db97fbb-d7d6-4cbe-bb8f-943e7c73606e', '5a1eabd7-112c-4895-9a36-7931c89b688e', '5d3006e4-6c25-4b87-b255-09adbd38a261', 'd81ec8c7-7911-4cdd-b7ad-879d6d11b465', 'b814bd90-d1a7-4e4b-94bc-c5a4a8476613', '9011c12e-191f-4a1f

In [11]:
node_info = index.docstore.get_node('2c2346d9-af03-4e52-a8c5-991adef40c59')
node_info

TextNode(id_='2c2346d9-af03-4e52-a8c5-991adef40c59', embedding=None, metadata={'license': 'nonexclusive-distrib'}, excluded_embed_metadata_keys=['license'], excluded_llm_metadata_keys=['license'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2211.08732v2.Lesion_Guided_Explainable_Few_Weak_shot_Medical_Report_Generation', node_type=None, metadata={'license': 'nonexclusive-distrib'}, hash='3e474263f62a4b69632e95f1638de432095b7560df9d06eeee3b332104cc6446'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='05dda7ee-5522-4052-a237-0807cae3bfd8', node_type=None, metadata={'license': 'nonexclusive-distrib'}, hash='37d4c6640d2a4f38792ebf86219da514af6acb7a55466500428b0e033833e18c')}, hash='a4b3eb091cab5018ef52d75c9bbbdd8051850618e9bd29ba1aa934f21de06775', text='Jinghan Sun1,Dong Wei2,Liansheng Wang1 ,and Yefeng Zheng2 Tencent Healthcare Shenzhen Co.LTD,Tencent Jarvis Lab,Shenzhen,China Abstract.Medical images are widely used in clinical practice for diagnosis.Au

In [12]:
ist = index.index_struct
#ist.table

In [13]:
def get_terms_for_node(index, node):
    terms = []
    for term in index.index_struct.table.keys():
        if(node in index.index_struct.table[term]):
            terms.append(term)
    return terms

terms = get_terms_for_node(index, '2c2346d9-af03-4e52-a8c5-991adef40c59')

In [14]:
terms

['Jinghan Sun', 'Liansheng Wang']

In [15]:
# this is time consuming for large indexes
# index.save_ref_doc_info()

doc_info = index.ref_doc_info

In [16]:
len(doc_info)

2808

In [17]:
import time
# get the first document
df = pd.DataFrame(columns=('doc_id', 'node_id', 'text', 'terms'))
i = 1
for doc_id in doc_info:
    doc = doc_info[doc_id]
    start = time.time()
    for node_id in doc.node_ids:
        terms = get_terms_for_node(index, node_id)
        node_info = index.docstore.get_node(node_id)
        #print(terms)
        df.loc[ len(df) ] = [doc_id, node_id, node_info.text, terms]
    end = time.time()
    print(f'{i}: {end - start}s - {doc_id}', end='\r')
    i = i+1
    #break

In [18]:
df.head()

,doc_id,node_id,text,terms
0,2205.10271v1.Compression_ensembles_quantify_ae...,6dfb1e49-578c-4f9a-a5b8-14f5a8a04d64,Compression ensembles quantify aesthetic compl...,"[visual, algorithmic, historical, contemporary..."
1,2205.10271v1.Compression_ensembles_quantify_ae...,7fca51b3-6b51-4029-80d0-de0a8912b07d,This approach is more efficient than other com...,"[This, more efficient, suited for]"
2,2205.10271v1.Compression_ensembles_quantify_ae...,13fca4a5-2173-4a8e-a7a5-63830337a393,Authors further quantify temporal resemblance ...,[]
3,2205.10271v1.Compression_ensembles_quantify_ae...,eb784e62-158a-486e-80b6-0c98781713cf,"quantification of visual aesthetics,including ...","[quantification of visual aesthetics, goes bac..."
4,2205.10271v1.Compression_ensembles_quantify_ae...,31a898e6-b6e8-4961-8a08-1d6a88594867,Palumbo et al.Guha and Ward Chamorro-Posada Ma...,"[colors, roundchrom, quantize, quantize3, colo..."


In [19]:
df.shape

(211378, 4)

In [20]:
import evaluate
rouge = evaluate.load('rouge')

def get_rouge_score(terms, text):
    #print(terms)
    #print(text)
    scores =  rouge.compute(predictions=[terms], references=[[text]])
    #print(scores)
    return scores


In [21]:
# calculate the rouge score
#df['rouge_score'] = df.apply(lambda x: get_rouge_score(' '.join(x['terms']), x['text']), axis=1)
df[["rouge1", "rouge2", "rougeL", "rougeLsum"]] = df.apply(lambda x: get_rouge_score(' '.join(x['terms']), x['text']), axis=1, result_type='expand')
df.head()

,doc_id,node_id,text,terms,rouge1,rouge2,rougeL,rougeLsum
0,2205.10271v1.Compression_ensembles_quantify_ae...,6dfb1e49-578c-4f9a-a5b8-14f5a8a04d64,Compression ensembles quantify aesthetic compl...,"[visual, algorithmic, historical, contemporary...",0.137255,0.020000,0.098039,0.117647
1,2205.10271v1.Compression_ensembles_quantify_ae...,7fca51b3-6b51-4029-80d0-de0a8912b07d,This approach is more efficient than other com...,"[This, more efficient, suited for]",0.092593,0.037736,0.092593,0.092593
2,2205.10271v1.Compression_ensembles_quantify_ae...,13fca4a5-2173-4a8e-a7a5-63830337a393,Authors further quantify temporal resemblance ...,[],0.000000,0.000000,0.000000,0.000000
3,2205.10271v1.Compression_ensembles_quantify_ae...,eb784e62-158a-486e-80b6-0c98781713cf,"quantification of visual aesthetics,including ...","[quantification of visual aesthetics, goes bac...",0.153846,0.112360,0.153846,0.153846
4,2205.10271v1.Compression_ensembles_quantify_ae...,31a898e6-b6e8-4961-8a08-1d6a88594867,Palumbo et al.Guha and Ward Chamorro-Posada Ma...,"[colors, roundchrom, quantize, quantize3, colo...",0.104348,0.053097,0.104348,0.104348


In [22]:
df.describe()

,rouge1,rouge2,rougeL,rougeLsum
count,211378.000000,211378.000000,211378.000000,211378.000000
mean,0.113045,0.054550,0.088802,0.092238
std,0.128184,0.085112,0.104879,0.107423
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.080000,0.017468,0.061224,0.064000
75%,0.175182,0.078431,0.132231,0.139535
max,1.000000,1.000000,1.000000,1.000000


In [23]:
df.to_csv(f"rouge-{index_model.replace('/', '-')}-default-no-coref.csv", index=False)